<a href="https://colab.research.google.com/github/ElizabethWaithera/AI-AGENTS/blob/main/L7_job_application_crew_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of embedchain[github,youtube] to determine which version is compatible with other requirements. This could take a while.


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew

**Note**:
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [4]:
import os
from google.colab import userdata


openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## crewAI Tools

In [7]:
!pip install pymupdf markdownify

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.5 MB/s eta 0:00:00


In [8]:
import fitz  # PyMuPDF
from markdownify import markdownify as md

def extract_text_from_pdf(file_path):
    # Open the PDF file
    pdf_document = fitz.open(file_path)
    text = ""

    # Iterate through each page and extract text
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()

    return text

def convert_text_to_markdown(text):
    # Convert text to Markdown using markdownify
    markdown_text = md(text)
    return markdown_text

file_path = '/content/ElizabethWaitheraThuoCV.pdf'
resume_text = extract_text_from_pdf(file_path)
markdown_resume = convert_text_to_markdown(resume_text)

# Save the Markdown content to a file
markdown_file_path = '/content/ElizabethWaitheraThuoCV.md'
with open(markdown_file_path, 'w') as f:
    f.write(markdown_resume)

print(f"Markdown file saved to {markdown_file_path}")


Markdown file saved to /content/ElizabethWaitheraThuoCV.md


In [9]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='/content/ElizabethWaitheraThuoCV.md')
semantic_search_resume = MDXSearchTool(mdx='/content/ElizabethWaitheraThuoCV.md')

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [10]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [11]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [12]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [13]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [14]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [15]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [16]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [17]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [18]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [19]:
job_application_inputs = {
    'job_posting_url': 'https://join.com/companies/kwanzatukule/13275591-data-analyst?pid=e65242534431eadcb0c9',
    'github_url': 'https://github.com/ElizabethWaithera',
    'personal_writeup': """Elizabeth is a passionate and ambitious data scientist and AI engineer  With two years of experience in data science and analytics, she has developed a robust skill set in machine learning, predictive modeling, and data visualization, leveraging tools like Python, R, SQL, and Microsoft Power BI. Her professional journey includes remote roles at Daada, Aquatrack, and Mastercard, where she built chatbots, developed machine learning algorithms, and created data pipelines to support operational strategies. As a Zindi Ambassador, Women in Data Science Ambassador, and Google Developer Group Organizer, Elizabeth actively promotes data science and fosters community learning. Her projects include developing a comprehensive stock analysis Streamlit app that integrates Alpha Vantage API and GPT-4 for sentiment analysis and conducting research on the relationship between women’s empowerment and HIV testing status. Driven by a strong analytical aptitude, Elizabeth is deeply interested in the intersection of finance trading and AI, continually seeking opportunities to innovate and solve real-world problems through data science."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [20]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://join.com/companies/kwanzatukule/13275591-data-analyst?pid=e65242534431eadcb0c9) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/ElizabethWaithera) URLs, and personal write-up (Elizabeth is a passionate and ambitious data scientist and AI engineer  With two years of experience in data science and analytics, she has developed a robust skill set in machine learning, predictive modeling, and data visualization, leveraging tools like Python, R, SQL, and Microsoft Power BI. Her professional journey includes remote roles at Daada, Aquatrack, and Mastercard,

- Dislplay the generated `tailored_resume.md` file.

In [21]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

ELIZABETH WAITHERA THUO
Elizabeththuo15@gmail.com Portfolio LinkedIn +254 740142019
EDUCATION AND LEADERSHIP
University of Nairobi, Nairobi, Kenya
September 2021 – May 2025
Bachelor of Economics and Statistics (GPA 3.8)
- Top 5% of class.
- Focus areas: Economic Theory, Statistical Methods, Quantitative and Analytical Skills, Applied Economics and Statistics, Research Methods,Interdisciplinary Applications
- Committee member: Students’ Consulting Group; sending out proposals and conducting cold calls to top management and strategy consulting firms.
ALX Africa
May 2023 – August 2024
Data Science (Overall Test Average: 91%)
- Awarded a $12,500 scholarship by the MasterCard Foundation to study the Data Science program at ALX Africa.
- Top 5% of cohort.
- Tools learned: Google Sheets, MySQL, Power BI, Python, Jupyter Notebooks.
edX
June 2024 – December 2024
Harvard University’s CS50x: Introduction to Computer Science
- Topics: Computer Science, Data Structures, SQL, Programming, Algorithms, Web Development, CSS, HTML, JavaScript.
Moi Girls’ School Nairobi, Nairobi, Kenya
January 2017 – April 2021
Kenya Certificate of Secondary Education (A-)
- Top of class.
- Captained the Volleyball team to the Kenya Secondary Schools Sports Association National games
WORK EXPERIENCE
Daada Inc. (Remote)
September 2023 – September 2024
Machine Learning and AI Engineer
- Presented findings and model performance metrics to stakeholders, resulting in the approval of a $500,000 budget for further AI initiatives, aimed at expanding the capabilities of machine learning applications across the organization.
- Conducted A/B testing on chatbot interactions, which demonstrated a 35% increase in conversion rates for sales inquiries, translating to an additional $200,000 in annual revenue
- Collaborated with cross-functional teams to integrate machine learning solutions into existing systems, leading to a 20% increase in process efficiency and a 15% reduction in time spent on manual data entry
Mastercard Foundation (Contract) September 2023 -February 2024
Machine Learning Engineer
- Utilized machine learning algorithms to analyze historical sales data and external factors, such as market trends and seasonal variations, which contributed to a 15% increase in sales during peak seasons. This data-driven approach allowed Naivas to better align its supply chain operations with customer demand.
- Collaborated with cross-functional teams, including marketing and supply chain management, to integrate forecasting insights into promotional strategies, leading to a 10% increase in promotional effectiveness and a 20% boost in customer engagement during targeted campaigns.
- Implemented a real-time analytics dashboard for monitoring demand trends and forecasting accuracy, which improved decision-making speed by 40%. This tool provided stakeholders with actionable insights, enabling quicker responses to market changes.
ALX AFRICA (Internship-Aquatrack) May 2024- August 2024
Data Scientist\Researcher
- Conducted market research and analysis to identify key trends and demands in the aquaculture industry, which informed the development of features that improved user engagement by 25%. This research helped tailor the tool to meet the specific needs of fish farmers.
- Integrated machine learning algorithms to provide personalized recommendations for farmers based on their production data and market trends.
ADDITIONAL INFORMATION
- Research assistant (under Davis Bundi Ntwiga; associate Professor and Research Scientist at the University of Nairobi).
- Community Organizer\Ambassador - Women In Data Science(WIDS), Google Developer Group(GDG), Zindi Data Science
- 2000+ hours of community service.
- Hackathon Finalist – Nation Media Group Hackathon, Inspire Network Hackathon
SKILLS
- AI & Machine Learning: OpenAI, Gemini, Hugging Face, LangChain, AWS, Google Cloud
- Data Visualization Tools: Tableau, Power BI, Matplotlib
- Programming Languages: Python, R, SQL, Java, C++
- Machine Learning: TensorFlow, Keras, Scikit-learn, Regression Analysis, Time Series Forecasting, A/B Testing
- Research Tools: Google Scholar, PubMed, Scopus, Mendeley, Zotero

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)